In [10]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [13]:
#oecd health spending 
spending_df = pd.read_csv('spending.csv')
#oecd life expectancy
expectancy_df = pd.read_csv('expectancy.csv')

In [17]:
expectancy_df

,VAR,Variable,UNIT,Measure,COU,Country,YEA,Year,Value,Flag Codes,Flags
0,EVIEFE00,Females at birth,EVIDUREV,Years,AUS,Australia,1970,1970,74.2,NaN,NaN
1,EVIEFE00,Females at birth,EVIDUREV,Years,AUS,Australia,1971,1971,74.9,NaN,NaN
2,EVIEFE00,Females at birth,EVIDUREV,Years,AUS,Australia,1972,1972,75.4,NaN,NaN
3,EVIEFE00,Females at birth,EVIDUREV,Years,AUS,Australia,1973,1973,75.5,NaN,NaN
4,EVIEFE00,Females at birth,EVIDUREV,Years,AUS,Australia,1974,1974,75.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
40303,EVIEHO00,Males at birth,EVIDUREV,Years,ARG,Argentina,2013,2013,72.9,NaN,NaN
40304,EVIEHO00,Males at birth,EVIDUREV,Years,ARG,Argentina,2014,2014,73.0,NaN,NaN
40305,EVIEHO00,Males at birth,EVIDUREV,Years,ARG,Argentina,2015,2015,73.4,NaN,NaN
40306,EVIEHO00,Males at birth,EVIDUREV,Years,ARG,Argentina,2016,2016,73.3,NaN,NaN


In [24]:
expectancy_df.rename(columns={'COU': 'LOCATION'}, inplace=True)


In [25]:
(spending_df.merge(expectancy_df, on = ['LOCATION', 'Year']))

,HF,Financing scheme,HC,Function,HP,Provider,MEASURE,Measure_x,LOCATION,Country_x,...,Flags_x,VAR,Variable,UNIT,Measure_y,Country_y,YEA,Value_y,Flag Codes_y,Flags_y
0,HFTOT,All financing schemes,HCTOT,Current expenditure on health (all functions),HPTOT,All providers,MLLNCU,Current prices,AUS,Australia,...,NaN,EVIEFE00,Females at birth,EVIDUREV,Years,Australia,1971,74.9,NaN,NaN
1,HFTOT,All financing schemes,HCTOT,Current expenditure on health (all functions),HPTOT,All providers,MLLNCU,Current prices,AUS,Australia,...,NaN,EVIEFE40,Females at age 40,EVIDUREV,Years,Australia,1971,37.4,NaN,NaN
2,HFTOT,All financing schemes,HCTOT,Current expenditure on health (all functions),HPTOT,All providers,MLLNCU,Current prices,AUS,Australia,...,NaN,EVIEFE60,Females at age 60,EVIDUREV,Years,Australia,1971,20.0,NaN,NaN
3,HFTOT,All financing schemes,HCTOT,Current expenditure on health (all functions),HPTOT,All providers,MLLNCU,Current prices,AUS,Australia,...,NaN,EVIEFE65,Females at age 65,EVIDUREV,Years,Australia,1971,16.1,NaN,NaN
4,HFTOT,All financing schemes,HCTOT,Current expenditure on health (all functions),HPTOT,All providers,MLLNCU,Current prices,AUS,Australia,...,NaN,EVIEFE80,Females at age 80,EVIDUREV,Years,Australia,1971,7.1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5727437,HF2,Voluntary health care payment schemes,HC7,Governance and health system and financing adm...,HPTOT,All providers,PPPPER,"Per capita, current prices, current PPPs",BRA,Brazil,...,NaN,EVIEHO00,Males at birth,EVIHFEEV,Difference male-female (years),Brazil,2017,-6.7,NaN,NaN
5727438,HF2,Voluntary health care payment schemes,HC7,Governance and health system and financing adm...,HPTOT,All providers,PPPPER,"Per capita, current prices, current PPPs",BRA,Brazil,...,NaN,EVIEHO40,Males at age 40,EVIHFEEV,Difference male-female (years),Brazil,2017,-5.1,NaN,NaN
5727439,HF2,Voluntary health care payment schemes,HC7,Governance and health system and financing adm...,HPTOT,All providers,PPPPER,"Per capita, current prices, current PPPs",BRA,Brazil,...,NaN,EVIEHO60,Males at age 60,EVIHFEEV,Difference male-female (years),Brazil,2017,-3.6,NaN,NaN
5727440,HF2,Voluntary health care payment schemes,HC7,Governance and health system and financing adm...,HPTOT,All providers,PPPPER,"Per capita, current prices, current PPPs",BRA,Brazil,...,NaN,EVIEHO65,Males at age 65,EVIHFEEV,Difference male-female (years),Brazil,2017,-3.2,NaN,NaN
